All essential modules from `labext` can be imported from `labext.prelude`.

In [1]:
import pandas as pd, numpy as np

from IPython.display import HTML, Javascript
from labext.prelude import M, A, W

%load_ext autoreload
%autoreload 2

## Javascript modules

The Javascript (JS) modules are accessed via the `M` namespace in `labext.prelude`. A module needs to be registered before using. As an example, we register the `DataTable` module to have a better display for pandas DataFrame.

In [2]:
M.DataTable.register()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

The `register` function will return a javascript code that modify your current HTML to inject required JS/CSS files (adding `script` and `link` tags). This function is safe to call repeatedly as it won't pollute your DOM.

You can also register multiple modules in one call using `M.register()` function

In [3]:
M.register([M.DataTable, M.Tippy])

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### DataTable

This JS module provides a better display for the pandas DataFrame. After the module is registered, all data frame will be converted automatically as shown in the example below.

Caveat: if your data frame contains thousands of rows, it will be very slow as DOM elements are created for all of the rows and return to the browser. This is the current limitation of this module. A not very ideal solution is to select top k rows in the data frame (e.g., `df.head(100)`).

In [4]:
df = pd.DataFrame([
    ['john', '123-231-2321', 30],
    ['peter', '231-231-2321', 35],
], columns=['name', 'phone', 'age'])
df

,name,phone,age
0,john,123-231-2321,30
1,peter,231-231-2321,35


By default, HTML content is escaped. You can change this behavior by setting arguments for this module as below.

In [5]:
M.DataTable.set_args(escape=False)

df = pd.DataFrame([
    ['<a href="http://google.com?q=john" style="color: blue" target="_blank">john</a>', '123-231-2321', 30]
], columns=['name', 'phone', 'age'])
display(df)

,name,phone,age
0,"<a href=""http://google.com?q=john"" style=""colo...",123-231-2321,30


### Tippy

This JS module allows you to add additional content that will show only when hovering over a DOM element. The additional content is added via the HTML attribute `data-tippy-content`, if it contains HTML, you can unescape it by setting `data-tippy-allowHTML` to be `true`.

Unlike the `DataTable` module. This JS module does not render the content automatically, you have to call `M.Tippy.render()` function after rendering the HTML as in the following example.

In [6]:
M.Tippy.register()

<IPython.core.display.Javascript object>

In [7]:
display(HTML('<button data-tippy-content="<b>Hello</b>, what are you doing?" data-tippy-allowHTML="true">Button</button>'))
M.Tippy.render()

<IPython.core.display.Javascript object>

Similar to the **DataTable** module, we can also configure the module for our need. For example, we may want the pop-over to not disappear so that we can interact with the content such as clicking the link.

All configurations are available on [**Tipply**'s website](https://atomiks.github.io/tippyjs/v6/all-props/).

In [8]:
M.Tippy.set_args(interactive=True, placement="right")

In [9]:
# we need the set the height of the container, as interactive Tippy's content is behind it, not like non-interactive content
popover = "<b>Hello</b>, please click to visit <a href='https://google.com' target='_blank'>google.com</a>"
display(HTML(f'<div style="height: 25px"><button data-tippy-content="{popover}" data-tippy-allowHTML="true">Button</button></div>'))
M.Tippy.render()

<IPython.core.display.Javascript object>

## Pre-built UI components

This library provides some UI components allowing us to interactive annotate/explore data. These components can be accessed via the `A` namespace.

### Slider

This component is for displaying an item in a list, and changing the item interactively. You need to provide a function that render the item given its index. The range of the slider is defined by two parameters: `min` (default 0) and `max` (inclusive).

In [10]:
lst = ["google.com", "amazon.com", "microsoft.com"]

def render(index):
    item = lst[index]
    display(HTML(f"<a href='https://{item}' target='_blank'>{item}</a>"))
    
A.slider(render, max=len(lst) - 1)

Output()

### Annotator

This component is for annotating a list of examples. Each example instance needs to have an `id` property (type `str`), and a `render` function to display the example. You can create a class that inherits the `A.annotators.Example` class to make sure that you implement the correct methods, and for python 3 type check to not complain, but it's completely optional.

Below is a simple example of classifying positive and negative reviews.

In [11]:
class MyExample:
    def __init__(self, id, review):
        self.review = review
        self.id = str(id)
        
    def render(self):
        display(HTML("<p>" + self.review + "</p>"))
        
reviews = ["It broke after two days", "It works as expected", "Good price"]
examples = [MyExample(i, review) for i, review in enumerate(reviews)]

This library provides basic annotators for the classification task: `A.annotators.ClassificationAnnotator` and `A.annotators.BatchClassificationAnnotator` (display multiple examples in one page). The annotated data is stored in a given file so you do not need to worry about losing your progress. The output file is in the CSV format, where the first column is the example id, and the second column is the label that you assigned.

If you need to customize these annotators (for example, assign multiple classes to one example), you can implement the abstract class `A.annotators.Annotator` or `A.annotators.PersistentAnnotator`. For more details, you can read the documentation in these classes.

In [12]:
outfile = "/tmp/labeled_data.csv"
annotator = A.annotators.ClassificationAnnotator(outfile, examples, ["positive", "negative"])
annotator.render()

Output(layout=Layout(border='1px solid #d24829', margin='0 0 0 -2px', padding='2px'))

In [13]:
outfile = "/tmp/labeled_data.csv"
annotator = A.annotators.BatchClassificationAnnotator(outfile, examples, ["positive", "negative"], batch_size=3)
annotator.render()

Output(layout=Layout(border='1px solid #d24829', margin='0 0 0 -2px', padding='2px'))

You can get the annotated result directly from the file, or from the `annotator.labeled_examples` property

In [14]:
annotator.labeled_examples

{}

In [15]:
!cat {outfile}

cat: /tmp/labeled_data.csv: No such file or directory


## Additional widgets

This library also provides some primitive widgets that you can use to build your own components. Those widgets can be imported via the `W` namespace.

A widget may require some JS modules, so you need to registered them as in the following examples:

In [16]:
M.register(W.HideableButton.required_modules())

### HideableButton

In [33]:
import ipywidgets as widgets

def render():    
    display(widgets.Button(description='another button'))
    
display(W.HideableButton('click to show', render, show_event='click').widget)

-----------